# Understanding X-ray Absorption Spectra by Means of Descriptors and Machine Learning Algorithms
### A. A. Guda, S. A. Guda, A. Martini, A.N. Kravtsova, A. Algasov, A. Bugaev, S. P. Kubrin, L. V. Guda, P. Šot, J. A. van Bokhoven, C. Copéret, A. V. Soldatov

The notebook contains the code for using machine learning algorithms and establishing the relationship between intuitive descriptors of spectra, such as edge position, intensities, positions and curvatures of minima and maxima on the one side, and those of the local atomic and electronic structure which are the coordination numbers, bond distances and angles, and oxidation state on the other. This approach allows overcoming the problem of the systematic difference between theoretical and experimental spectra. Furthermore, the numerical relations can be expressed in analytical formulas providing a simple and fast tool to extract structural parameters based on spectral shape. The methodology was successfully applied to experimental data of the multicomponent Fe:SiO2 system and reference iron compounds, demonstrating the high prediction quality for both theoretical validation sets and experimental data.

### Current folder contains:
- exp subfolder with experimental spectra
- generated subfolder with data generated during calculations and used afterwards
- results subfolder
- samples subfolder with theoretical spectra database
- xyz subfolder with .xyz files used in molecule constructor and samples generation
- exp_true_values.csv file with known parameters for experimental spectra database
- instructions.docx file with install instructions
- paper_calculations.ipynb - this notebook
- Fe_project.py - project file with different settings

## Import [PyFitIt](http://hpc.nano.sfedu.ru/pyfitit)

In [ ]:
from pyfitit import *
import pandas as pd

resultFolder = 'results'

## Plot spectra of the calculated samples (Figure 4a)

In [ ]:
for cn in range(2,7):
    sample = readSample(f'samples/sample_{cn}')
    proj = loadProject('Fe_project.py', CN=cn, valence=3)
    sample.spectra = smoothLib.smoothDataFrame(proj.FDMNES_smooth, sample.spectra, 'fdmnes', 
                                               proj.spectrum, proj.intervals['fit_norm'], 
                                               folder=sample.folder, norm=proj.FDMNES_smooth['norm'])
    sample.limit(energyRange=[7100, 7200], inplace=True)
    plotting.plotSample(sample.energy, sample.spectra, color_param=sample.params['r1'], 
                        fileName=f'{resultFolder}/sample_adaptive_CN{cn}_wo_pre-edge.png', 
                        title=f'Sample for cn={cn}', alpha=0.1)

## Print information about structure parameters of the samples

### Relation between paper notation p1,p2,p3,... and sampled parameters:

- for cn=2: p1=r1, p2=r1+d12, p3=phi1, p4=psi
- for cn=3: p1=phi, p2=r1, p3=r2, p4=function(p1)
- for cn=4: p1=phi1, p2=phi2, p3=r1, p4=r2
- for cn=5: p1=r2, p2=r2+d2, p3=r1, p4=phi2, p5=phi1
- for cn=6: p1=r2, p2=r2+d2, p3=r1, p4=phi2, p5=phi1

In [ ]:
cn = 2
sample = readSample(f'samples/sample_{cn}')
sample.params.describe()

In [ ]:
sample.params

## Construct database from samples for different cn and calculate descriptors of structure

In [ ]:
# calculate structure parameters: mean and standard deviation of distances to O atoms
def calcDist(params, mol, CN):
    dists = mol.getSortedDists('O')
    return [np.mean(dists[:CN]), np.std(dists[:CN])]

# read pre-calculated samples and replace individual structure parameters by common: cn, valence, mean and std of distances to O atoms
# These samples were calculated by adaptive sampling.
# To get uniformly distributed geometry parameters, the samples can be converted to IHS by interpolation when flag convertToIHS=True
def loadSpectra(energyRange=None, sampleFolder='samples', convertToIHS=False):
    all_data = None
    for CN in range(2,7):
        for valence in [2,3]:
            data = readSample(sampleFolder+os.sep+f'sample_{CN}')
            projFile = 'Fe_project.py'
            project = loadProject(projFile, CN=CN, valence=valence)
            if convertToIHS:
                geometryParamRanges = project.geometryParamRanges
                for r in ['r1', 'r2']:
                    if r in geometryParamRanges: geometryParamRanges[r][0] = 1.8
                data = sampling.convertSampleTo(data, 'IHS', 500, geometryParamRanges, seed=0)
                data.saveToFolder(f'generated/samples/sample_{CN}_IHS_generated')
            n = len(data.params)
            oldParams = data.paramNames
            data.addParam(paramGenerator=lambda params, mol: calcDist(params, mol, CN), paramName=['FeODist','stdDist'], project=project)
            data.addParam(paramName='CN', paramData=np.ones(n)*CN)
            data.addParam(paramName='valence', paramData=np.ones(n)*valence)
            data.delParam(oldParams)
            data.spectra = smoothLib.smoothDataFrame(project.FDMNES_smooth, data.spectra, 'fdmnes', project.spectrum, project.intervals['fit_norm'], folder=data.folder, norm=project.FDMNES_smooth['norm'])
            assert np.all(data.spectra.values<15)
            data.addParam(paramName='name', paramData=np.array([f'cn{CN}v{valence}_{i}' for i in range(n)], dtype=object))
            if all_data is None: all_data = data
            else: all_data.unionWith(data)

    # take all experimental spectra from folder exp
    exp_spectra_names = []
    exp_files = os.listdir('exp')
    trueParams = pd.read_csv(f'exp_true_values.csv', sep=';')
    for f in exp_files:
        name = os.path.splitext(os.path.split(f)[-1])[0]
        exp_spectra_names.append(name)
        sp = readSpectrum(f'exp{os.sep}{f}')
        i = np.where(trueParams['name'].to_numpy() == name)[0]
        if len(i) > 0:
            assert len(i) == 1
            i = i[0]
            true = {col:trueParams.loc[i,col] for col in set(trueParams.columns) if col == 'name' or not np.isnan(trueParams.loc[i,col])}
        else: true = {'name': name}
        all_data.addRow(sp, true)

    all_data.limit(energyRange)
    return all_data, exp_spectra_names

if os.path.exists(f'generated/data_initial.pkl'):
    singleComponentData, exp_spectra_names = utils.load_pkl(f'generated/data_initial.pkl')
    singleComponentDataIHSgen, _ = utils.load_pkl(f'generated/data_initial_IHS_generated.pkl')
else:
    energyRange = [7100, 7200]
    singleComponentData, exp_spectra_names = loadSpectra(energyRange, convertToIHS=False)
    singleComponentDataIHSgen, _ = loadSpectra(energyRange, convertToIHS=True)
    utils.save_pkl((singleComponentData, exp_spectra_names), f'generated/data_initial.pkl')
    utils.save_pkl((singleComponentDataIHSgen, exp_spectra_names), f'generated/data_initial_IHS_generated.pkl')


## Plot experimental spectra

In [ ]:
print(exp_spectra_names)

In [ ]:
toPlotNames = ['FeIII siloxide', 'Darwin glass', 'Zhamanshinite-3', 'akermanite', 'Fe@SiO2_1', 'kirschsteinite']
toPlot = tuple()
for name in toPlotNames:
    sp = readSpectrum(f'exp{os.sep}{name}.txt')
    toPlot += (sp.energy, sp.intensity, name)
plotting.plotToFile(*toPlot, fileName='results/exp_spectra.png', xlim=[7100, 7200])

## Calculate descriptors of spectra

In [ ]:
def calc_descriptors(sample, usePcaPrebuildData, pcaPrebuildDataFile):
    efermi = {'type':'efermi', 'columnName': 'Edge'}
    stableMin = {'type': 'stableExtrema', 'extremaType': 'min', 'energyInterval': [7135,7190], 
                 'plotFolderPrefix': 'generated/stable_extrema', 'columnName': 'Pit'}
    stableMax = copy.deepcopy(stableMin)
    stableMax['extremaType'] = 'max'
    stableMax['energyInterval'] = [7120,7150]
    stableMax['columnName'] = 'WL'
    relPcaPrebuildDataFile = os.path.split(pcaPrebuildDataFile)[0]+os.sep+'rel_'+os.path.split(pcaPrebuildDataFile)[-1]
    sample, goodSpectrumIndices = descriptor.addDescriptors(sample, [stableMin, stableMax, efermi, 
         {'type':'pca', 'usePcaPrebuildData':usePcaPrebuildData, 'fileName':pcaPrebuildDataFile, 'columnName': 'PCA'}, 
         {'type':'rel_pca', 'usePcaPrebuildData':usePcaPrebuildData, 'fileName':relPcaPrebuildDataFile, 'columnName': 'rPCA'}])
    d = sample.params
    sample.addParam(paramName='Pit_e-WL_e', paramData=d['Pit_e'] - d['WL_e'])
    sample.addParam(paramName='WL-Pit_slope', paramData=(d['WL_i'] - d['Pit_i'])/(d['Pit_e'] - d['WL_e']))
    return sample, goodSpectrumIndices

# if descriptors were already calculated we just load it
if os.path.exists(f'generated/data.pkl'):
    singleComponentData, _ = utils.load_pkl(f'generated/data.pkl')
    singleComponentDataIHSgen, _ = utils.load_pkl(f'generated/data_IHS_gen.pkl')
else:
    singleComponentData, _ = calc_descriptors(singleComponentData, usePcaPrebuildData=False, pcaPrebuildDataFile='generated/pca_data.pkl')
    singleComponentDataIHSgen, _ = calc_descriptors(singleComponentDataIHSgen, usePcaPrebuildData=False, pcaPrebuildDataFile='generated/pca_data_IHS_gen.pkl')
    utils.save_pkl((singleComponentData, exp_spectra_names), f'generated/data.pkl')
    utils.save_pkl((singleComponentDataIHSgen, exp_spectra_names), f'generated/data_IHS_gen.pkl')
    
# divide sample into two parts: known (theory) and unknown (experiments)
singleComponentData, exp_sample = singleComponentData.splitUnknown()
singleComponentDataIHSgen, _ = singleComponentDataIHSgen.splitUnknown()

## Plot descriptors (Figure 5)

In [ ]:
for combination in [['WL_d2', 'Pit_e'], ['Edge_e', 'WL_d2'], ['Edge_e', 'Pit_e'], ['rPCA2', 'rPCA3'], ['WL_e', 'Pit_e-WL_e'], ['PCA2', 'rPCA3']]:
    descriptor.plot_descriptors_2d(singleComponentData.params, combination, ['CN'], folder_prefix=f'{resultFolder}/scatter_plots', 
                                   unknown=exp_sample.params, markersize=50, textsize=0, alpha=0.3, plot_only=True, doNotPlotRemoteCount=6)
for combination in [['WL_e', 'Pit_e-WL_e'], ['PCA2', 'rPCA3']]:
    descriptor.plot_descriptors_2d(singleComponentData.params, combination, ['valence'], folder_prefix=f'{resultFolder}/scatter_plots', 
                                   unknown=exp_sample.params, markersize=50, textsize=0, alpha=0.3, plot_only=True, doNotPlotRemoteCount=6)

## Find best descriptor subsets of size 1, 2, 3, 4 (Table 3)

In [ ]:
# the quality is measured using cv_count cross-validation technique, which is repeated m times
# quality = accuracy for classification, r2-score - for regression
def getQuality(label_name, combination):
    print('Try to predict', label_name, 'by', combination)
    qualityResult = descriptor.getQuality(singleComponentDataIHSgen.params, combination, [label_name], cv_count=10, m=5, printDebug=False)
    print(f"quality = {qualityResult[label_name]['quality']:.3f}±{qualityResult[label_name]['quality_std']:.3f}")

getQuality(label_name='CN', combination=['Pit_i'])
getQuality(label_name='CN', combination=['WL_i'])
getQuality(label_name='CN', combination=['Edge_e'])
getQuality(label_name='CN', combination=['WL_d2'])
getQuality(label_name='CN', combination=['PCA3'])
getQuality(label_name='CN', combination=['WL-Pit_slope'])

getQuality(label_name='CN', combination=['Edge_e', 'WL_d2'])
getQuality(label_name='CN', combination=['Edge_e', 'Edge_slope'])
getQuality(label_name='CN', combination=['WL_d2', 'Pit_e'])

getQuality(label_name='CN', combination=['Edge_e', 'WL_e', 'rPCA3'])
getQuality(label_name='CN', combination=['Edge_e', 'Edge_slope', 'WL_d2'])

getQuality(label_name='CN', combination=['Edge_e', 'Edge_slope', 'WL_e', 'Pit_e'])

# ===================================================

getQuality(label_name='FeODist', combination=['Pit_e'])
getQuality(label_name='FeODist', combination=['Pit_e-WL_e'])
getQuality(label_name='FeODist', combination=['Edge_slope'])
getQuality(label_name='FeODist', combination=['WL_e'])
getQuality(label_name='FeODist', combination=['rPCA2'])
getQuality(label_name='FeODist', combination=['PCA3'])

getQuality(label_name='FeODist', combination=['rPCA2', 'rPCA3'])
getQuality(label_name='FeODist', combination=['WL_d2', 'rPCA3'])
getQuality(label_name='FeODist', combination=['Edge_e', 'Pit_e'])

getQuality(label_name='FeODist', combination=['PCA2', 'PCA3', 'rPCA3'])
getQuality(label_name='FeODist', combination=['Edge_e', 'WL_e', 'Pit_e'])

getQuality(label_name='FeODist', combination=['Edge_e', 'WL_e', 'Pit_d2', 'Pit_e'])

# =====================================================================

getQuality(label_name='stdDist', combination=['Pit_i'])
getQuality(label_name='stdDist', combination=['Pit_d2'])
getQuality(label_name='stdDist', combination=['Edge_e'])
getQuality(label_name='stdDist', combination=['rPCA3'])
getQuality(label_name='stdDist', combination=['rPCA2'])
getQuality(label_name='stdDist', combination=['PCA3'])

getQuality(label_name='stdDist', combination=['WL_d2', 'Pit_i'])
getQuality(label_name='stdDist', combination=['Pit_i', 'rPCA2'])
getQuality(label_name='stdDist', combination=['PCA3', 'Pit_i'])

getQuality(label_name='stdDist', combination=['Edge_e', 'WL_d2', 'Pit_i'])
getQuality(label_name='stdDist', combination=['PCA2', 'PCA3', 'Pit_i'])

getQuality(label_name='stdDist', combination=['Edge_e', 'WL_d2', 'WL_i', 'Pit_i'])

# =====================================================================

getQuality(label_name='valence', combination=['PCA1'])
getQuality(label_name='valence', combination=['Edge_e'])
getQuality(label_name='valence', combination=['WL_e'])
getQuality(label_name='valence', combination=['PCA2'])
getQuality(label_name='valence', combination=['rPCA1'])
getQuality(label_name='valence', combination=['PCA3'])

getQuality(label_name='valence', combination=['PCA1', 'rPCA3'])
getQuality(label_name='valence', combination=['Edge_e', 'WL_i'])
getQuality(label_name='valence', combination=['WL_e', 'Pit_e'])

getQuality(label_name='valence', combination=['Edge_e', 'WL_d2', 'Pit_e-WL_e'])
getQuality(label_name='valence', combination=['PCA2', 'rPCA2', 'rPCA3'])

getQuality(label_name='valence', combination=['Edge_e', 'WL_e', 'Pit_e-WL_e', 'Pit_i'])

## Analytical relations between descriptors of spectra and descriptors of structure (Table 4)

In [ ]:
label_names = ['CN', 'FeODist', 'stdDist', 'valence']
features = sorted(list(set(singleComponentDataIHSgen.paramNames) - set(label_names) - {'PCA1','PCA2','PCA3','rPCA1','rPCA2','rPCA3','name'}))
descriptor.getAnalyticFormulasForGivenFeatures(singleComponentDataIHSgen.params, features, label_names, l1_ratio=1, 
                                               output_file=f'{resultFolder}/analytic_labels_by_features.txt')
descriptor.getAnalyticFormulasForGivenFeatures(singleComponentDataIHSgen.params, sorted(list(set(label_names) - {'valence'})), features, 
                                               output_file=f'{resultFolder}/analytic_features_by_labels.txt')

## Building sample of mixtures

In [ ]:
# Do not rebuild sample, if it's saved copy exits
if os.path.exists(f'generated/mix_data.pkl'):
    mixtureData = utils.load_pkl(f'generated/mix_data.pkl')
    mixtureDataIHSgen = utils.load_pkl(f'generated/mix_data_IHS_generated.pkl')
else:
    # generate mixtures of random spectra from singleComponentData with random concentrations
    # target features (labels) are calculated as weighted average
    label_names = ['CN', 'FeODist', 'stdDist', 'valence']
    mixtureData = mixture.generateMixtureOfSample(size=5000, componentCount=2, sample=singleComponentData, 
        label_names=label_names, addDescrFunc=lambda sample: calc_descriptors(sample,True,'generated/pca_data.pkl'), 
        randomSeed=1, componentNameColumn='name')
    mixtureDataIHSgen = mixture.generateMixtureOfSample(size=5000, componentCount=2, sample=singleComponentDataIHSgen, 
        label_names=label_names, addDescrFunc=lambda sample: calc_descriptors(sample,True,'generated/pca_data_IHS_gen.pkl'), 
        randomSeed=1, componentNameColumn='name')
    utils.save_pkl(mixtureData, f'generated/mix_data.pkl')
    utils.save_pkl(mixtureDataIHSgen, f'generated/mix_data_IHS_generated.pkl')
    mixtureData.params.to_csv(f'generated/mix_data.csv', index=False, sep=' ')
    mixtureDataIHSgen.params.to_csv(f'generated/mix_data_IHS_generated.csv', index=False, sep=' ')

## Scatter plots of descriptors calculated for mixtures (Figure 8)

In [ ]:
descriptor.plot_descriptors_2d(mixtureData.params, ['WL_d2', 'Pit_e'], ['CN'], folder_prefix=f'{resultFolder}/scatter_mix_plots', 
                               unknown=exp_sample.params, markersize=50, textsize=0, alpha=0.3, plot_only=True, doNotPlotRemoteCount=0)
descriptor.plot_descriptors_2d(mixtureData.params, ['WL_d2', 'Pit_e'], ['FeODist'], folder_prefix=f'{resultFolder}/scatter_mix_plots', 
                               unknown=exp_sample.params, markersize=50, textsize=0, alpha=0.3, plot_only=True, doNotPlotRemoteCount=0)
descriptor.plot_descriptors_2d(mixtureData.params, ['WL_e', 'Pit_e-WL_e'], ['valence'], folder_prefix=f'{resultFolder}/scatter_mix_plots', 
                               unknown=exp_sample.params, markersize=50, textsize=0, alpha=0.3, plot_only=True, doNotPlotRemoteCount=0)

## Comparison between cross-validation quality calculated for selected combinations of descriptors (Tables 5,6 and Figure 9)

In [ ]:
# Table 5
table = 5
descriptor.descriptor_quality(mixtureData.params, ['CN'], ['Edge_e', 'Pit_e-WL_e', 'WL_d2'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_CN')
descriptor.descriptor_quality(mixtureData.params, ['FeODist'], ['Edge_e', 'WL_e', 'Pit_e'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_FeODist')
descriptor.descriptor_quality(mixtureData.params, ['valence'], ['Edge_e', 'Pit_e', 'WL_i'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_valence')

# Table 6 and Figure 9
table = 6
descriptor.descriptor_quality(mixtureData.params, ['CN'], ['WL_e', 'Pit_e', 'rPCA2'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_CN')
descriptor.descriptor_quality(mixtureData.params, ['FeODist'], ['WL_i', 'Pit_i', 'rPCA2'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_FeODist')
descriptor.descriptor_quality(mixtureData.params, ['valence'], ['Edge_e', 'WL_e', 'PCA3'], feature_subset_size=3, cv_parts_count=10, cv_repeat=3, 
                              unknown_data=exp_sample.params, textColumn='name', folder=f'{resultFolder}/table_{table}_valence')

## Relative to constant prediction error calculated for training set of 600 spectra for CN=6 (Table S1)

In [ ]:
v = 3;    cn = 6
method = 'RBF'  # 'Ridge', 'Ridge Quadric', 'Extra Trees', 'RBF' 'LightGBM'
proj = loadProject('Fe_project.py', CN=cn, valence=v)
# this function builds ML model for spectra predition by structure parameters
estimator = constructInverseEstimator(method, proj, proj.FDMNES_smooth, CVcount=10, 
                                      folderToSaveCVresult=f'{resultFolder}/{method}_v{v}_cn{cn}_CV')
sample = readSample('samples/sample_'+str(cn))
estimator.fit(sample)